## CAR Model for crash prediction
### Developed by: bpben

In [2]:
import re
import csv
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy.stats as ss
from glob import glob
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.stats import describe

In [3]:
import json

In [18]:
adj = pd.read_csv('../../data_generation/data/adjacency_info.csv', dtype={'segment_id':'str'})

In [6]:
import pymc3 as pm

### Data processing
The approach here is to create 3 time-lag features:

1. crashes in the past week
2. crashes in the past month
3. crashes in the past quarter (three months)
4. average crashes per week up to target week

All features except 4 are calculated to exclude one another.  That is, crashes in the past month does not include the past week's crashes.  Crashes in the past quarter do not include the past month.

In [7]:
SEG_CHARS = ['AADT', 'SPEEDLIMIT', 'Struct_Cnd', 'Surface_Tp', 'F_F_Class']

In [8]:
# Read in data
data = pd.read_csv('../../data_gen/data/vz_predict_dataset.csv.gz', compression='gzip', dtype={'segment_id':'str'})
data.sort_values(['segment_id', 'week'], inplace=True)

In [9]:
# get segments with non-zero crashes
data_nonzero = data.set_index('segment_id').loc[data.groupby('segment_id').crash.sum()>0]
data_nonzero.reset_index(inplace=True)

In [10]:
def format_crash_data(data, col, target):
    """ formats crash data for train/test 
    target: week to predict (make into binary target)
        must be >4 months in
    gets previous week count, previous month count, previous quarter count, avg per week
    
    """
    assert target>16
    pre_week = target - 1
    pre_month = range(pre_week-4, target)
    pre_quarter = range(pre_month[0]-12, target)
    all_prior_weeks = range(1, target)
    
    # week interval for each segment
    # full range = pre_quarter : target
    sliced = data.loc[(slice(None),slice(1, target)),:]
    week_data = sliced[col].unstack(1)
    week_data.reset_index(level=1, inplace=True)
    
    # aggregate
    week_data['pre_month'] = week_data[pre_month].sum(axis=1)
    week_data['pre_quarter'] = week_data[pre_quarter].sum(axis=1)
    week_data['pre_week'] = week_data[pre_week]
    # avg as of target week
    week_data['avg_week'] = week_data[all_prior_weeks].apply(
        lambda x: x.sum() / len(all_prior_weeks), axis=1
    )
    
    # binarize target
    week_data['target'] = (week_data[target]>0).astype(int)
    
    return(week_data[['segment_id','target', 'pre_week', 
                      'pre_month', 'pre_quarter', 'avg_week']])

In [11]:
# arbitrarily choosing week = 50
crash_lags = format_crash_data(data_nonzero.set_index(['segment_id','week']), 'crash', 50)

In [12]:
data_segs = data_nonzero.groupby('segment_id')[SEG_CHARS].max()
data_segs.reset_index(inplace=True)

In [13]:
data_model = crash_lags.merge(data_segs, on='segment_id')

In [21]:
# add adjacency information
data_model = data_model.merge(adj, on='segment_id')

In [52]:
#pd.get_dummiesadj.set_index('segment_id').unstack()
adj_mat = adj.merge(adj, on='orig_id')
# drop self in list
adj_mat = adj_mat[adj_mat.segment_id_x!=adj_mat.segment_id_y]
# get adj in the form needed for pymc3 implementtion
adj_mat = adj_mat.groupby('segment_id_x').segment_id_y.unique()

In [28]:
# prepare data
N = len(data_model) # number of observations
O = data_model.target
adj.groupby('segment_id').orig_id.unique()

segment_id
0                                   [994764, 994765, 997749]
000                                                    [990]
001                                                    [991]
0010                                                  [9910]
00100                                                [99115]
001000                                              [991099]
0010000                                            [9911468]
0010001                                            [9911469]
0010002                                            [9911470]
0010003                                            [9911471]
0010004                                            [9911472]
0010005                                            [9911478]
0010006                                            [9911479]
0010007                                            [9911480]
0010008                                            [9911482]
0010009                                            [9911483]
001001       

In [24]:
class CAR(pm.distributions.distribution.Continuous):
    """
    Conditional Autoregressive (CAR) distribution

    Parameters
    ----------
    a : list of adjacency information
    w : list of weight information
    tau : precision at each location
    """
    def __init__(self, w, a, tau, *args, **kwargs):
        super(CAR, self).__init__(*args, **kwargs)
        self.a = a = tt.as_tensor_variable(a)
        self.w = w = tt.as_tensor_variable(w)
        self.tau = tau*tt.sum(w, axis=1)
        self.mode = 0.

    def get_mu(self, x):

        def weigth_mu(w, a):
            a1 = tt.cast(a, 'int32')
            return tt.sum(w*x[a1])/tt.sum(w)

        mu_w, _ = scan(fn=weigth_mu,
                       sequences=[self.w, self.a])

        return mu_w

    def logp(self, x):
        mu_w = self.get_mu(x)
        tau = self.tau
        return tt.sum(continuous.Normal.dist(mu=mu_w, tau=tau).logp(x))

In [25]:
with pm.Model() as model1:
    # Vague prior on intercept
    beta0 = pm.Normal('beta0', mu=0.0, tau=1.0e-5)
    # Vague prior on covariate effect
    beta1 = pm.Normal('beta1', mu=0.0, tau=1.0e-5)

    # Random effects (hierarchial) prior
    tau_h = pm.Gamma('tau_h', alpha=3.2761, beta=1.81)
    # Spatial clustering prior
    tau_c = pm.Gamma('tau_c', alpha=1.0, beta=1.0)

    # Regional random effects
    theta = pm.Normal('theta', mu=0.0, tau=tau_h, shape=N)
    mu_phi = CAR('mu_phi', w=wmat, a=amat, tau=tau_c, shape=N)

    # Zero-centre phi
    phi = pm.Deterministic('phi', mu_phi-tt.mean(mu_phi))

    # Mean model
    mu = pm.Deterministic('mu', tt.exp(logE + beta0 + beta1*aff + theta + phi))

    # Likelihood
    Yi = pm.Poisson('Yi', mu=mu, observed=O)

    # Marginal SD of heterogeniety effects
    sd_h = pm.Deterministic('sd_h', tt.std(theta))
    # Marginal SD of clustering (spatial) effects
    sd_c = pm.Deterministic('sd_c', tt.std(phi))
    # Proportion sptial variance
    alpha = pm.Deterministic('alpha', sd_c/(sd_h+sd_c))

NameError: name 'N' is not defined

In [13]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], look_back, trainX.shape[2]))
testX = np.reshape(testX, (testX.shape[0], look_back, testX.shape[2]))

In [19]:
import keras.backend as K

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

In [56]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(10, input_shape=trainX.shape[1:]))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(trainX, trainY,
          validation_data=(testX, testY),
          epochs=10, batch_size=1000, verbose=2)

Train on 70225 samples, validate on 34583 samples
Epoch 1/10
5s - loss: 0.3952 - acc: 0.9978 - val_loss: 0.1353 - val_acc: 0.9992
Epoch 2/10
3s - loss: 0.0782 - acc: 0.9993 - val_loss: 0.0351 - val_acc: 0.9992
Epoch 3/10
3s - loss: 0.0268 - acc: 0.9993 - val_loss: 0.0187 - val_acc: 0.9992
Epoch 4/10
4s - loss: 0.0172 - acc: 0.9993 - val_loss: 0.0153 - val_acc: 0.9992
Epoch 5/10
4s - loss: 0.0140 - acc: 0.9993 - val_loss: 0.0132 - val_acc: 0.9992
Epoch 6/10
4s - loss: 0.0120 - acc: 0.9993 - val_loss: 0.0117 - val_acc: 0.9992
Epoch 7/10
4s - loss: 0.0106 - acc: 0.9993 - val_loss: 0.0106 - val_acc: 0.9992
Epoch 8/10
3s - loss: 0.0096 - acc: 0.9993 - val_loss: 0.0098 - val_acc: 0.9992
Epoch 9/10
4s - loss: 0.0089 - acc: 0.9993 - val_loss: 0.0092 - val_acc: 0.9992
Epoch 10/10
4s - loss: 0.0083 - acc: 0.9993 - val_loss: 0.0087 - val_acc: 0.9992


In [57]:
# make predictions
trainPredict = model.predict_proba(trainX)
testPredict = model.predict_proba(testX)

34400/34583 [============================>.] - ETA: 0s

In [54]:
a = testY - testPredict

In [35]:
np.sum(testY - np.reshape(testPredict, (a.shape[0],)))

-374.35548718459904

In [36]:
testPredict.shape

(34583, 1)

In [49]:
from sklearn.metrics import roc_auc_score

In [58]:
roc_auc_score(testY, np.reshape(testPredict, (a.shape[0],)))

0.39403292797081224